# Trustworthy AI Explainer Dashboard: Interactive UI for LLM Applications
**Module 15 Team Project**

---

**Team Name:** Group4

**Team Members:**
1. Member A (PM/Integrator)
2. Member B (Gradio)
3. Member C (Streamlit)
4. Member D (Explainability)
5. Member E (Deployment/Docs)


## Team planning and role assignment

**Team discussion: Decide how your team will divide the work and collaborate.**

**Suggested roles (adapt as needed):**
- **Gradio developer:** Lead rapid prototyping and Gradio interface development
- **Streamlit architect:** Design and implement Streamlit dashboard structure
- **Backend integrator:** Connect LLM, explainability, and feedback systems
- **Deployment specialist:** Handle deployment, testing, and documentation

**Your team's role distribution:**

| Team Member | Primary Role | Secondary Responsibilities |
|------------|--------------|----------------------------|
| | | |
| | | |
| | | |
| | | |

**Application focus:** ___________________________

**Target users:** ___________________________

**Collaboration approach:** ___________________________

**Chosen use case (simple, no RAG):** Tutor/Explainer chatbot for Trustworthy AI concepts (memory + explainability + feedback).

**Explainability plan:** LIME (local token influence on input) + SHAP (tabular feature contributions for a lightweight quality score).

**Deployment plan:** Gradio → Hugging Face Spaces, Streamlit → Streamlit Community Cloud.


## Step 1: Environment setup

Install required libraries for UI development, LLM integration, and explainability.

**Note:** This notebook is for learning and prototyping. Your final deliverables will be:
1. `gradio_app_template.py` - Gradio prototype
2. `streamlit_app_template.py` - Streamlit dashboard

These template files are provided in your project directory.


In [ ]:
# Install required libraries (uncomment to install)
# !pip install gradio streamlit
# !pip install openai langchain langchain-openai langchain-community
# !pip install pandas plotly
# !pip install shap lime  # For explainability
# !pip install sentence-transformers  # For embeddings

# Import libraries
import os
import time
from typing import List, Dict, Tuple, Optional

print("✅ Libraries ready!")
print("\nUI Frameworks:")
print("  - Gradio: For rapid prototyping")
print("  - Streamlit: For dashboards")
print("\nLLM Options:")
print("  - OpenAI API")
print("  - AWS Bedrock")
print("  - Local Ollama")
print("  - Hugging Face Transformers")
print("\nExplainability:")
print("  - SHAP (global)")
print("  - LIME (local)")


## Part 1: Gradio rapid prototyping

### Understanding Gradio

**What is Gradio?**

Gradio is a Python library for quickly creating web interfaces for your machine learning models and functions.

**Key advantages:**
- Wrap any Python function in a web UI (minutes, not hours)
- Built-in components for ML tasks (chat, file upload, etc.)
- Native integration with Hugging Face Spaces
- Share via public link instantly
- Perfect for demos and proof-of-concepts

**Architecture pattern:**
```
Your Python Function → gr.Interface(fn=...) → Web Application
```

**When to use Gradio:**
- Quick prototypes and demos
- Sharing single functions/models
- PoC for stakeholders
- Hugging Face deployment


In [ ]:
# Gradio Example 1: Simple Interface
import gradio as gr

def text_analyzer(text: str, analysis_type: str) -> dict:
    """Analyze text and return statistics."""
    return {
        "Character count": len(text),
        "Word count": len(text.split()),
        "Analysis type": analysis_type,
        "Sample": text[:50] + "..." if len(text) > 50 else text
    }

# Create Gradio interface
demo_simple = gr.Interface(
    fn=text_analyzer,
    inputs=[
        gr.Textbox(label="Enter text", lines=5, placeholder="Type or paste text here..."),
        gr.Radio(choices=["Basic", "Advanced"], label="Analysis Type", value="Basic")
    ],
    outputs=gr.JSON(label="Results"),
    title="Text Analyzer Demo",
    description="Simple Gradio interface demonstrating input/output mapping"
)

print("✅ Simple Gradio interface created!")
print("To run: demo_simple.launch()")
print("\n💡 Key insight: Gradio wraps your function with minimal code!")


### Gradio chatbot interface

**gr.ChatInterface** - Pre-built component for conversational AI

**Key features:**
- Automatically manages conversation history
- Built-in retry, undo, clear buttons
- Example prompts for users
- Streaming support
- Custom styling

**Function signature:**
```python
def chat_fn(message: str, history: List[Tuple[str, str]]) -> str:
    # message: current user message
    # history: list of (user_msg, bot_msg) tuples
    # return: bot response
```


In [ ]:
# Gradio Example 2: Chatbot with Memory
import gradio as gr
import time

def chatbot_response(message: str, history: list) -> str:
    """
    Generate chatbot response with context awareness.
    
    Args:
        message: Current user message
        history: List of (user_msg, bot_msg) tuples
    
    Returns:
        Bot response string
    """
    # Simulate processing time
    time.sleep(0.5)
    
    # Context-aware response
    num_exchanges = len(history)
    
    # Mock LLM response (replace with actual LLM)
    if "hello" in message.lower():
        response = f"Hello! This is exchange #{num_exchanges + 1}."
    elif "history" in message.lower():
        response = f"We've had {num_exchanges} exchanges so far."
    elif num_exchanges > 0:
        last_topic = history[-1][0][:30]
        response = f"You said: '{message}'. Earlier you asked about: '{last_topic}...'"
    else:
        response = f"You said: '{message}'. How can I help?"
    
    return response

# Create chatbot interface
chat_demo = gr.ChatInterface(
    fn=chatbot_response,
    title="Chatbot with Memory Demo",
    description="Try asking about the conversation history!",
    examples=["Hello!", "What did I say before?", "Tell me about history"],
    retry_btn="🔄 Retry",
    undo_btn="↩️ Undo",  
    clear_btn="🗑️ Clear History"
)

print("✅ Gradio chatbot created!")
print("To run: chat_demo.launch()")
print("\n💡 History is automatically managed by Gradio!")


## Part 2: Streamlit dashboard development

### Understanding Streamlit

**What is Streamlit?**

Streamlit turns Python scripts into interactive web applications.

**Key differences from Gradio:**

| Feature | Gradio | Streamlit |
|---------|---------|-----------|
| **Focus** | Function wrapping | Full applications |
| **Complexity** | Simple demos | Complex dashboards |
| **Layout** | Limited | Highly flexible |
| **State** | Implicit | Explicit (`session_state`) |
| **Best for** | Quick prototypes | Production apps |

**Streamlit's reactive model:**
1. User interacts (button, input, etc.)
2. Entire script re-runs top-to-bottom
3. UI updates with new values
4. State persists via `st.session_state`

**Critical concept: Caching**

Since scripts re-run on every interaction, expensive operations must be cached:
- `@st.cache_resource`: For models, connections (non-serialized objects)
- `@st.cache_data`: For data loading, computations (serializable data)


In [ ]:
# Streamlit Example: State Management Pattern
# Note: This is pseudo-code for learning - see streamlit_app_template.py for working code

streamlit_state_example = """
import streamlit as st

# Initialize session state (runs once per session)
if 'counter' not in st.session_state:
    st.session_state.counter = 0

if 'messages' not in st.session_state:
    st.session_state.messages = []

# Display current state
st.write(f"Counter: {st.session_state.counter}")
st.write(f"Messages: {len(st.session_state.messages)}")

# Buttons that modify state
if st.button("Increment Counter"):
    st.session_state.counter += 1
    st.rerun()  # Trigger re-run with new state

# Text input
if message := st.text_input("Add message"):
    st.session_state.messages.append(message)
    # State automatically persists

# Display all messages
for i, msg in enumerate(st.session_state.messages):
    st.write(f"{i+1}. {msg}")
"""

print("📝 Streamlit State Management Pattern:")
print(streamlit_state_example)
print("\n✅ Key concepts:")
print("  - Initialize state on first run")
print("  - Access via st.session_state")
print("  - State persists across reruns")
print("  - Use st.rerun() when needed")


### Streamlit caching strategies

**Why caching matters:**

Without caching, expensive operations run on every user interaction:
- Loading models (10+ seconds)
- Computing embeddings (seconds per call)
- API calls (cost + latency)
- Database queries

**Two caching decorators:**

**@st.cache_resource** - For non-serializable objects
```python
@st.cache_resource
def load_model():
    return HuggingFaceModel("model-name")  # Loads once, reused forever
```

Use for:
- ML models
- Database connections
- API clients
- Thread pools

**@st.cache_data** - For serializable data
```python
@st.cache_data(ttl=3600)  # Cache for 1 hour
def load_data():
    return pd.read_csv("data.csv")  # Cached by input params
```

Use for:
- DataFrames
- Lists, dicts
- Computation results
- API responses (with TTL)


In [ ]:
# Streamlit Caching Example
caching_example = """
import streamlit as st
import time

# BAD: Without caching (runs every time)
def expensive_computation_bad(x):
    time.sleep(3)  # Simulates expensive operation
    return x * 2

# GOOD: With caching (runs once per unique input)
@st.cache_data
def expensive_computation_good(x):
    time.sleep(3)  # Only runs first time for each x
    return x * 2

# Model loading example
@st.cache_resource
def load_llm():
    # Loads once, cached forever
    print("Loading model...")  # Only prints once
    return MockLLM()

# Usage
st.title("Caching Demo")

# First call: takes 3 seconds
# Subsequent calls with same input: instant
number = st.number_input("Enter number", value=5)
result = expensive_computation_good(number)
st.write(f"Result: {result}")

# Model loads once for entire session
model = load_llm()
"""

print("📝 Streamlit Caching Example:")
print(caching_example)
print("\n✅ Performance impact:")
print("  - Without caching: 3s per interaction")
print("  - With caching: 3s first time, <1ms after")
print("  - 3000x speedup!")


## Part 3: Team project implementation

### Your deliverables

Your team will create two complete applications:

**1. Gradio Prototype (`Group4_Module15_GradioApp.py`)**
- Simple chatbot interface
- LLM integration
- Basic explainability display
- User feedback mechanism (thumbs up/down)
- Ready to deploy to Hugging Face Spaces

**2. Streamlit Dashboard (`Group4_Module15_StreamlitApp.py`)**
- Multi-page application:
  - Page 1: Chat interface with memory
  - Page 2: Explainability analysis
  - Page 3: Feedback dashboard
  - Page 4: System monitoring
  - Page 5: Documentation
- State management for conversation history
- Caching for expensive operations
- Feedback collection and visualization
- Performance metrics tracking

### Implementation guidance

**Templates provided:**

Both application templates are provided in your project directory with:
- Complete working code structure
- Placeholder functions for LLM integration
- Mock implementations for testing
- Detailed comments and TODOs
- Deployment instructions

**Your team should:**

1. **Customize the templates** for your use case
2. **Integrate actual LLM** (OpenAI, Bedrock, Ollama, or Transformers)
3. **Implement explainability** (SHAP or LIME visualizations)
4. **Test thoroughly** with different inputs and edge cases
5. **Deploy** to appropriate platform (Hugging Face Spaces or Streamlit Cloud)
6. **Document** your customizations and decisions


In [ ]:
# Verify template files exist
import os

template_files = [
    "Group4_Module15_GradioApp.py",
    "Group4_Module15_StreamlitApp.py"
]

print("🔍 Checking for template files...\n")

for file in template_files:
    if os.path.exists(file):
        size = os.path.getsize(file)
        print(f"✅ {file} (Found: {size:,} bytes)")
    else:
        print(f"❌ {file} NOT FOUND")

print("\n📝 To run the applications:")
print("  Gradio:    python Group4_Module15_GradioApp.py")
print("  Streamlit: streamlit run Group4_Module15_StreamlitApp.py")
print("\n💡 Both templates are fully functional with mock LLM!")
print("   Customize them for your team's project.")


## Part 4: Deployment guide

### Deployment options

**1. Hugging Face Spaces (Gradio)**

Best for: Quick demos, public sharing

```bash
# Setup
git init
echo "gradio\nopenai" > requirements.txt
git add gradio_app_template.py requirements.txt
git commit -m "Initial commit"

# Deploy
# 1. Create new Space on huggingface.co
# 2. Choose "Gradio" as SDK
# 3. Push to the Space repo
git remote add hf https://huggingface.co/spaces/YOUR_USERNAME/YOUR_SPACE
git push hf main
```

**Important:** Use Hugging Face Secrets for API keys!

**2. Streamlit Community Cloud**

Best for: Team dashboards, internal tools

```bash
# Setup
git init  
echo "streamlit\npandas\nplotly" > requirements.txt
git add streamlit_app_template.py requirements.txt
git commit -m "Initial commit"
git push origin main

# Deploy
# 1. Go to share.streamlit.io
# 2. Connect your GitHub repo
# 3. Select branch and file
# 4. Deploy!
```

**Important:** Add secrets in Streamlit Cloud settings!

**3. Local Docker**

Best for: Development, self-hosted

```dockerfile
# Dockerfile for Streamlit
FROM python:3.11-slim
WORKDIR /app
COPY requirements.txt .
RUN pip install -r requirements.txt
COPY streamlit_app_template.py .
EXPOSE 8501
CMD ["streamlit", "run", "streamlit_app_template.py"]
```

```bash
docker build -t my-ai-dashboard .
docker run -p 8501:8501 my-ai-dashboard
```

### Security best practices

**Never hardcode API keys:**

```python
# WRONG
openai.api_key = "sk-abc123..."

# RIGHT - Environment variables
import os
openai.api_key = os.getenv("OPENAI_API_KEY")

# RIGHT - Streamlit secrets
import streamlit as st
openai.api_key = st.secrets["OPENAI_API_KEY"]
```

**Secrets configuration:**

**Hugging Face Spaces:**
- Settings → Repository secrets
- Add `OPENAI_API_KEY = your_key`
- Access via `os.getenv()`

**Streamlit Cloud:**
- App settings → Secrets
- Add TOML format:
```toml
OPENAI_API_KEY = "your_key"
```
- Access via `st.secrets["OPENAI_API_KEY"]`


## Team project summary

### A. Team information

**Team Name:** Group4

**Application Name:** Trustworthy AI Tutor (Explainable)

**Target Use Case:** Tutor/Explainer chatbot for Trustworthy AI (no RAG) with memory, LIME/SHAP, feedback and monitoring.

**Target Users:** ___________________________

### B. Gradio prototype summary

**Completed features:**
- [ ] Basic interface with inputs/outputs
- [ ] LLM integration
- [ ] Chatbot with conversation history
- [ ] Explainability display
- [ ] User feedback mechanism
- [ ] Deployed to Hugging Face Spaces

**LLM choice:** ___________________________

**Key customizations made:** ___________________________

**Deployment URL:** ___________________________

### C. Streamlit dashboard summary

**Completed features:**
- [ ] Multi-page structure
- [ ] Chat interface with memory
- [ ] State management for history
- [ ] Caching for expensive operations
- [ ] Explainability analysis page
- [ ] Feedback collection and visualization
- [ ] Performance monitoring
- [ ] Documentation page
- [ ] Deployed to Streamlit Cloud

**Pages implemented:**
1. ___________________________
2. ___________________________
3. ___________________________
4. ___________________________
5. ___________________________

**State management approach:** ___________________________

**Caching strategy:** ___________________________

**Deployment URL:** ___________________________

### D. Explainability integration

**Method used:** [ ] SHAP [ ] LIME [ ] Both

**Implementation details:** ___________________________

**Visualizations created:** ___________________________

**User value provided:** ___________________________

### E. Feedback mechanism

**Feedback type:** ___________________________

**Storage approach:** ___________________________

**Analytics implemented:** ___________________________

**Total feedback collected (during testing):** ___________________________

### F. Team collaboration

**Role distribution:**

| Team Member | Primary Role | Key Contributions |
|------------|--------------|-------------------|
| | | |
| | | |
| | | |
| | | |

**Collaboration tools used:** ___________________________

**Challenges encountered:** ___________________________

**Solutions implemented:** ___________________________

### G. Technical decisions

**Framework comparison insights:**

**When to use Gradio:**
- ___________________________
- ___________________________

**When to use Streamlit:**
- ___________________________
- ___________________________

**Most valuable feature learned:** ___________________________

**Most challenging aspect:** ___________________________

### H. Deployment experience

**Platform(s) used:** ___________________________

**Deployment challenges:** ___________________________

**Security measures implemented:** ___________________________

**Performance observations:** ___________________________

### I. Future improvements

**Short-term enhancements:**
1. ___________________________
2. ___________________________
3. ___________________________

**Long-term vision:**
___________________________

**Scalability considerations:**
___________________________

### J. Learning outcomes

**Key concepts mastered:**
- [ ] Gradio interface development
- [ ] Streamlit dashboard creation
- [ ] State management in reactive apps
- [ ] Caching strategies for performance
- [ ] Multi-page application architecture
- [ ] Deployment to cloud platforms
- [ ] API key security best practices
- [ ] User feedback collection
- [ ] Explainability integration

**Most valuable insight:** ___________________________

**How this applies to real projects:** ___________________________

**Skills gained:** ___________________________


## Team submission checklist

Before submitting, ensure your team has completed:

**Planning and setup:**
- [ ] Team roles clearly defined and documented
- [ ] Application use case and target users identified
- [ ] Development environment configured
- [ ] All required libraries installed

**Gradio prototype:**
- [ ] `gradio_app_template.py` customized for your use case
- [ ] LLM integrated (OpenAI/Bedrock/Ollama/Transformers)
- [ ] Chatbot interface functional with conversation memory
- [ ] Explainability display implemented
- [ ] User feedback mechanism (thumbs up/down) working
- [ ] Tested with multiple example conversations
- [ ] Deployed to Hugging Face Spaces (or local deployment documented)
- [ ] API keys properly secured (no hardcoded secrets)

**Streamlit dashboard:**
- [ ] `streamlit_app_template.py` customized for your use case
- [ ] Multi-page structure implemented
- [ ] Chat page with conversation history working
- [ ] State management implemented for persistence
- [ ] Caching applied to expensive operations
- [ ] Explainability analysis page functional
- [ ] Feedback dashboard with visualizations
- [ ] Monitoring page showing metrics
- [ ] Documentation page completed with team information
- [ ] Tested thoroughly (state, caching, all pages)
- [ ] Deployed to Streamlit Cloud (or local deployment documented)
- [ ] API keys secured via secrets management

**Explainability integration:**
- [ ] SHAP or LIME implemented
- [ ] Visualizations display correctly
- [ ] Explanations integrated with LLM responses
- [ ] Users can understand model decisions

**Documentation:**
- [ ] Team information documented
- [ ] Role distribution explained
- [ ] Technical decisions justified
- [ ] Deployment instructions provided
- [ ] User guide included
- [ ] Code well-commented

**Testing:**
- [ ] Both applications tested with various inputs
- [ ] Edge cases handled (empty input, long text, errors)
- [ ] Performance acceptable (caching working)
- [ ] Feedback mechanism collecting data properly
- [ ] Deployment working (accessible URLs)

**Submission files:**
- [ ] `TeamName_Module15_Project.ipynb` (this completed notebook)
- [ ] `TeamName_Module15_GradioApp.py` (customized Gradio app)
- [ ] `TeamName_Module15_StreamlitApp.py` (customized Streamlit app)
- [ ] `requirements.txt` (all dependencies listed)
- [ ] `README.md` (deployment and usage instructions)
- [ ] `deployment_urls.txt` (links to deployed applications)

**Estimated completion time: 80 minutes (per team member)**

---

## Resources and next steps

**Official documentation:**
- Gradio: https://www.gradio.app/docs
- Streamlit: https://docs.streamlit.io/
- Hugging Face Spaces: https://huggingface.co/docs/hub/spaces
- Streamlit Community Cloud: https://streamlit.io/cloud

**Deployment platforms:**
- Hugging Face Spaces: https://huggingface.co/spaces
- Streamlit Cloud: https://share.streamlit.io
- Docker Hub: https://hub.docker.com/

**Explainability libraries:**
- SHAP: https://shap.readthedocs.io/
- LIME: https://lime-ml.readthedocs.io/

**LLM integration:**
- OpenAI: https://platform.openai.com/docs
- LangChain: https://python.langchain.com/docs
- Hugging Face Transformers: https://huggingface.co/docs/transformers

---

**End of Module 15 Team Project**

**Good luck with your Trustworthy AI Explainer Dashboard!** 
